
# Gas Turbine Emissions Prediction (CO & NOx)

**Modernized implementation using:**
- Scikit-learn Pipelines
- GridSearchCV
- StandardScaler
- MLPRegressor
- Clean train/validation/test split


In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import optuna, shap, warnings, tensorflow as tf
from xgboost import XGBRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import Ridge
from sklearn.isotonic import IsotonicRegression
from sklearn.metrics import mean_absolute_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input

warnings.filterwarnings('ignore')
sns.set_theme(style="whitegrid")

ModuleNotFoundError: No module named 'pandas'

# 1. DATA INGESTION & TEMPORAL SPLIT (Preventing Autocorrelation Leakage)

In [7]:

def load_and_split():
    files = [f"gt_{y}.csv" for y in range(2011, 2016)]
    data = [pd.read_csv(f) for f in files]
    
    # Per Heybak et al.: Train (11-12), Val (13), Test (14-15)
    train_df = pd.concat(data[0:2])
    val_df = data[2]
    test_df = pd.concat(data[3:5])
    
    feat = ['AT', 'AP', 'AH', 'AFDP', 'GTEP', 'TIT', 'TAT', 'TEY', 'CDP']
    return train_df[feat], train_df['NOX'], val_df[feat], val_df['NOX'], test_df[feat], test_df['NOX'], feat

X_train, y_train, X_val, y_val, X_test, y_test, feat_names = load_and_split()

scaler = StandardScaler()
X_train_s = scaler.fit_transform(X_train)
X_val_s = scaler.transform(X_val)
X_test_s = scaler.transform(X_test)


NameError: name 'pd' is not defined

# 2. BAYESIAN OPTIMIZATION (Optuna)


In [ ]:
def objective(trial):
    params = {
        'hidden_layer_sizes': (trial.suggest_int('u1', 64, 128), trial.suggest_int('u2', 32, 64)),
        'alpha': trial.suggest_float('alpha', 1e-4, 1e-1, log=True),
        'learning_rate_init': trial.suggest_float('lr', 1e-4, 1e-2, log=True)
    }
    model = MLPRegressor(**params, max_iter=200, random_state=42).fit(X_train_s, y_train)
    return model.score(X_val_s, y_val)

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=15)

# 3. HYBRID STACKING ENSEMBLE (MLP + XGBoost)


In [ ]:
best_mlp = MLPRegressor(hidden_layer_sizes=(study.best_params['u1'], study.best_params['u2']), 
                        alpha=study.best_params['alpha'], random_state=42)

stacking_pems = StackingRegressor(
    estimators=[('mlp', best_mlp), ('xgb', XGBRegressor(n_estimators=200, max_depth=5))],
    final_estimator=Ridge(alpha=1.0)
).fit(X_train_s, y_train)

# 4. ISOTONIC CALIBRATION & UNCERTAINTY (Platt Scaling Equivalent for Regression)
# Predict Intervals (Quantile Regression)


In [ ]:
upper_model = XGBRegressor(objective='reg:quantileerror', quantile_alpha=0.95).fit(X_train_s, y_train)
lower_model = XGBRegressor(objective='reg:quantileerror', quantile_alpha=0.05).fit(X_train_s, y_train)

# Calibrate Mean Output

In [ ]:
iso_calib = IsotonicRegression(out_of_bounds='clip').fit(stacking_pems.predict(X_val_s), y_val)
final_preds = iso_calib.predict(stacking_pems.predict(X_test_s))

# 5. DEEP LEARNING (LSTM for Sequential Memory)

In [ ]:
def create_seq(X, y, steps=5):
    X_s, y_s = [], []
    for i in range(len(X)-steps):
        X_s.append(X[i:(i+steps)])
        y_s.append(y.iloc[i+steps])
    return np.array(X_s), np.array(y_s)

X_train_seq, y_train_seq = create_seq(X_train_s, y_train)
X_test_seq, y_test_seq = create_seq(X_test_s, y_test)

lstm_pems = Sequential([
    Input(shape=(5, 9)),
    LSTM(50, return_sequences=True),
    LSTM(25),
    Dense(1)
])
lstm_pems.compile(optimizer='adam', loss='mae')
lstm_pems.fit(X_train_seq, y_train_seq, epochs=10, batch_size=64, verbose=0)

# 6. SOTA VISUALIZATION & XAI (SHAP)

In [ ]:
print(f"PEMS Stacking R2: {r2_score(y_test, final_preds):.4f}")

# Uncertainty Plot

In [ ]:
plt.figure(figsize=(12, 5))
plt.fill_between(range(100), lower_model.predict(X_test_s[:100]), upper_model.predict(X_test_s[:100]), alpha=0.2, label='90% Confidence (PEMS Uncertainty)')
plt.plot(final_preds[:100], label='Predicted NOx (Calibrated)')
plt.scatter(range(100), y_test[:100], color='red', s=10, label='Actual Sensor')
plt.title("SOTA PEMS: Prediction with Uncertainty Intervals")
plt.legend()
plt.show()

# SHAP Explainability

In [ ]:
explainer = shap.Explainer(stacking_pems.predict, X_train_s[:100])
shap_values = explainer(X_test_s[:300])
shap.summary_plot(shap_values, X_test_s[:300], feature_names=feat_names)